In [1]:
!pwd

/gpfs/projects/kernlab/stittes/analysis2/notebooks


In [2]:
import glob
from datetime import datetime
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import stdpopsim
import re

def extract_timestamps(file_string):
    full_matches = re.findall(r'\[[A-Za-z]{3} [A-Za-z]{3} \d{2} \d{2}:\d{2}:\d{2} \d{4}\]', file_string)
    return full_matches


#content = "INFO:snakemake.logging:[Tue Jun 24 09:57:26 2025]"
#extract_timestamps(content)

def get_runtimes(file_paths):
    """
    Get the runtimes in the Snakemake logs for a list of paths.
    Returns a list of runtimes in minutes.
    """
    diff_min_list = {}
    for path in file_paths:
        diff_min = np.nan
        id_str = "Empty"
        fh = open(path, mode="r")
        times_list = []
        for line in fh:
            time_stamps = extract_timestamps(line)
            if len(time_stamps) > 1:
                print(f"Log {path} has more than one timestamp in line: {line.strip()}")
                continue
            if "wildcards" in line:
                id_str = line.split("wildcards: ")[1].split("seed=")[0].strip()
                if id_str not in diff_min_list:
                    diff_min_list[id_str] = []
                continue
            if len(time_stamps) == 0:
                continue
            times_list.append(datetime.strptime(time_stamps[0].replace("[", "").replace("]", ""), "%a %b %d %H:%M:%S %Y"))
        if len(times_list) > 1:
            diff_min = (times_list[-1] - times_list[0]).total_seconds()/60
            diff_min_list[id_str].append(diff_min)
        else:
            print(f"Log {path} has only one timestamp, cannot calculate runtime.")
    return diff_min_list

In [3]:
logs = glob.glob("/gpfs/projects/kernlab/stittes/analysis2_runtimes/logs/simulation_simulation/*")
logtime_dict = get_runtimes(logs)
logtime_dict

{'demog=OutOfAfricaArchaicAdmixture_5R19, dfes=Gamma_K17, annots=all_sites, seeds=1675701734, chrms=chr16': [916.6],
 'demog=OutOfAfricaArchaicAdmixture_5R19, dfes=none, annots=all_sites, seeds=1675701734, chrms=chr6': [27.183333333333334],
 'demog=Constant, dfes=Gamma_K17, annots=all_sites, seeds=1845187043, chrms=chr1': [4843.033333333334],
 'demog=Constant, dfes=none, annots=all_sites, seeds=1358822686, chrms=chr1': [72.18333333333334],
 'demog=OutOfAfricaArchaicAdmixture_5R19, dfes=Gamma_K17, annots=ensembl_havana_104_exons, seeds=1845187043, chrms=chr15': [53.55],
 'demog=Constant, dfes=none, annots=all_sites, seeds=1845187043, chrms=chr22': [30.666666666666668],
 'demog=Constant, dfes=Gamma_K17, annots=all_sites, seeds=1675701734, chrms=chr11': [2083.7],
 'demog=Constant, dfes=Gamma_K17, annots=ensembl_havana_104_exons, seeds=1358822686, chrms=chr7': [205.21666666666667],
 'demog=Constant, dfes=Gamma_K17, annots=all_sites, seeds=1845187043, chrms=chr20': [1469.3],
 'demog=OutOfAf

In [4]:
species = stdpopsim.get_species("HomSap")
names, lengths = zip(*[(f"chr{i}", species.get_contig(f"chr{i}").interval_list[0][0][1]) for i in range(1, 23)])
chrom_size_df = pd.DataFrame({"chrms": list(names), "lengths": list(lengths)})

In [5]:
data_dict = {}
for outer_key, outer_value in logtime_dict.items():
    sub_keys = outer_key.split(",")
    for s in sub_keys:
        sub_dict_string = s.strip().split("=")
        if len(sub_dict_string) < 2:
            continue
        key, value = sub_dict_string
        if key not in data_dict:
            data_dict[key] = []
        data_dict[key].append(value)
    
    if len(outer_value) == 1:
        time = outer_value[0]
    else:
        time = "0"
    if 'time' not in data_dict:
        data_dict['time'] = []  
    data_dict['time'].append(time)

times_df = pd.DataFrame(data_dict).merge(chrom_size_df, on = "chrms")
times_df["species"] = ["HomSap" if "chr" in i else "PhoSin" for i in times_df['chrms']]
display(times_df)
print(len(times_df[times_df['time'] > 0]))


,demog,dfes,annots,seeds,chrms,time,lengths,species
0,OutOfAfricaArchaicAdmixture_5R19,Gamma_K17,all_sites,1675701734,chr16,916.600000,90338345,HomSap
1,OutOfAfricaArchaicAdmixture_5R19,none,all_sites,1675701734,chr6,27.183333,170805979,HomSap
2,Constant,Gamma_K17,all_sites,1845187043,chr1,4843.033333,248956422,HomSap
3,Constant,none,all_sites,1358822686,chr1,72.183333,248956422,HomSap
4,OutOfAfricaArchaicAdmixture_5R19,Gamma_K17,ensembl_havana_104_exons,1845187043,chr15,53.550000,101991189,HomSap
...,...,...,...,...,...,...,...,...
391,OutOfAfricaArchaicAdmixture_5R19,none,all_sites,1358822686,chr2,34.650000,242193529,HomSap
392,OutOfAfricaArchaicAdmixture_5R19,none,all_sites,1358822686,chr5,30.550000,181538259,HomSap
393,OutOfAfricaArchaicAdmixture_5R19,Gamma_K17,all_sites,1358822686,chr5,1476.133333,181538259,HomSap
394,OutOfAfricaArchaicAdmixture_5R19,Gamma_K17,ensembl_havana_104_exons,1675701734,chr2,123.166667,242193529,HomSap


396


In [6]:
from lets_plot import *
LetsPlot.setup_html()

subset_times_df = times_df.query("(dfes == 'Gamma_K17' & annots == 'ensembl_havana_104_exons') | (dfes == 'none' & annots == 'all_sites')").reset_index()
subset_times_df['chrom_rank'] = [int(c.replace("chr", ""))for c in subset_times_df['chrms']]
display(subset_times_df)
(ggplot(subset_times_df, aes(x = 'time')) +
    geom_histogram() +
    geom_density() +
    facet_grid("demog", scales = "free") +\
    scale_color_discrete()
)

,index,demog,dfes,annots,seeds,chrms,time,lengths,species,chrom_rank
0,1,OutOfAfricaArchaicAdmixture_5R19,none,all_sites,1675701734,chr6,27.183333,170805979,HomSap,6
1,3,Constant,none,all_sites,1358822686,chr1,72.183333,248956422,HomSap,1
2,4,OutOfAfricaArchaicAdmixture_5R19,Gamma_K17,ensembl_havana_104_exons,1845187043,chr15,53.550000,101991189,HomSap,15
3,5,Constant,none,all_sites,1845187043,chr22,30.666667,50818468,HomSap,22
4,7,Constant,Gamma_K17,ensembl_havana_104_exons,1358822686,chr7,205.216667,159345973,HomSap,7
...,...,...,...,...,...,...,...,...,...,...
259,389,OutOfAfricaArchaicAdmixture_5R19,none,all_sites,1358822686,chr14,18.333333,107043718,HomSap,14
260,390,Constant,Gamma_K17,ensembl_havana_104_exons,1358822686,chr19,226.900000,58617616,HomSap,19
261,391,OutOfAfricaArchaicAdmixture_5R19,none,all_sites,1358822686,chr2,34.650000,242193529,HomSap,2
262,392,OutOfAfricaArchaicAdmixture_5R19,none,all_sites,1358822686,chr5,30.550000,181538259,HomSap,5


In [7]:
# Define your desired order
demog_order = ['OutOfAfricaArchaicAdmixture_5R19', 'Constant']  # replace with your actual values

# Convert to categorical with specific order
subset_times_df['demog'] = pd.Categorical(subset_times_df['demog'], 
                                         categories=demog_order, 
                                         ordered=True)

#swap names in dfes from 'Gamma_K17' to 'BGS' and 'none' to 'Neutral'
subset_times_df['dfes'] = subset_times_df['dfes'].replace({'Gamma_K17': 'BGS', 'none': 'Neutral'})
# same for demog
#subset_times_df = subset_times_df.set_index('demog').loc[demog_order].reset_index()


In [10]:
runtime_plot = (ggplot(subset_times_df, aes(x = 'lengths', y = "time", shape = "dfes")) +
   geom_point() +
    facet_grid("demog", "dfes", scales = "free") +
    ylab("Time (minutes)") +
    xlab("Chromosome lengths (base pairs)") +
    scale_color_brewer(name='Simulation type',
                         labels=['Neutral', 'BGS']) +
    theme(legend_position='none')
)

ggsave(runtime_plot, "runtime_plot.pdf", path = ".")

'/gpfs/projects/kernlab/stittes/analysis2/notebooks/runtime_plot.pdf'

In [9]:
!pwd

/gpfs/projects/kernlab/stittes/analysis2/notebooks
